In [ ]:
# Cargar la tabla completa
combined_reviews = pd.read_sql("SELECT * FROM combined_reviews", con=engine)

# Filtrar en Python
filtered_reviews = combined_reviews.groupby('Modelo estandar').filter(lambda x: x['comentario'].nunique() >= 200)

# Guardar la tabla filtrada nuevamente en MySQL
filtered_reviews.to_sql('filtered_reviews', con=engine, if_exists='replace', index=False)